<a href="https://colab.research.google.com/github/pnakinprabhakar/week6/blob/main/week6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task: File Ingestion and Schema validation
Take any csv/text file of 2+ GB of your choice. --- (You can do this assignment on Google colab)

Read the file ( Present approach of reading the file )

Try different methods of file reading eg: Dask, Modin, Ray, pandas and present your findings in term of computational efficiency

Perform basic validation on data columns : eg: remove special character , white spaces from the col name

As you already know the schema hence create a YAML file and write the column name in YAML file. --define separator of
read and write file, column name in YAML

Validate number of columns and column name of ingested file with YAML.

Write the file in pipe separated text file (|) in gz format.

Create a summary of the file:

Total number of rows,

total number of columns

file size

In [ ]:
import os 
import time

In [ ]:
! pip install dask

You should consider upgrading via the 'c:\pyhton37\python.exe -m pip install --upgrade pip' command.


In [ ]:
from dask import dataframe as dd

In [ ]:
start = time.time()
dask_df = dd.read_csv("imdb.csv")
end = time.time()
print('Read csv with dask = ',(end-start),'sec')

Read csv with dask =  0.03203392028808594 sec


In [ ]:
import pandas as pd
start = time.time()
df = pd.read_csv('imdb.csv')
end = time.time()
print("Read csv with Pandas",(end-start),"sec")

Read csv with Pandas 0.008001089096069336 sec


Validation

In [ ]:
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

In [ ]:
%%writefile testutility.py



def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)



def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing testutility.py


In [ ]:
%%writefile movie.yaml
file_type: csv
dataset_name: test
file_name: imdb
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
        - rank
        - name
        - year
        - rating
        - genre
        - certificate
        - run_time
        - tagline
        - budget
        - box_office
        - casts
        - directors
        - writers

Overwriting file.yaml


In [ ]:
!pip install PyYAML

You should consider upgrading via the 'c:\pyhton37\python.exe -m pip install --upgrade pip' command.


In [ ]:
!pip install utility 

  Created wheel for utility: filename=utility-1.0-py3-none-any.whl size=3831 sha256=47ce7069a3e6b8bdbc4d1bcde87c3128b97987c1c6f7d9ea60e7e905a38772a1
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\5c\f4\ee\6ac70a2b3218653fb6229f5f1157ac819a05f0419f4ac898fa
Successfully built utility


You should consider upgrading via the 'c:\pyhton37\python.exe -m pip install --upgrade pip' command.


In [ ]:
import yaml
import testutility as util


In [ ]:
config_data = util.read_config_file("file.yaml")

In [ ]:
config_data['inbound_delimiter']

','

In [ ]:
config_data

{'file_type': 'csv',
 'dataset_name': 'test',
 'file_name': 'imdb',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['rank',
  'name',
  'year',
  'rating',
  'genre',
  'certificate',
  'run_time',
  'tagline',
  'budget',
  'box_office',
  'casts',
  'directors',
  'writers']}

In [ ]:
file_type = config_data['file_type']
source_file = "./"+config_data['file_name'] + f'.{file_type}'
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

c:\pyhton37\lib\site-packages\IPython\core\interactiveshell.py:3398: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,rank,name,year,rating,genre,certificate,run_time,tagline,budget,box_office,casts,directors,writers
0,1,The Shawshank Redemption,1994,9.3,Drama,R,2h 22m,Fear can hold you prisoner. Hope can set you f...,25000000,28884504,"Tim Robbins,Morgan Freeman,Bob Gunton,William ...",Frank Darabont,"Stephen King,Frank Darabont"
1,2,The Godfather,1972,9.2,"Crime,Drama",R,2h 55m,An offer you can't refuse.,6000000,250341816,"Marlon Brando,Al Pacino,James Caan,Diane Keato...",Francis Ford Coppola,"Mario Puzo,Francis Ford Coppola"
2,3,The Dark Knight,2008,9.0,"Action,Crime,Drama",PG-13,2h 32m,Why So Serious?,185000000,1006234167,"Christian Bale,Heath Ledger,Aaron Eckhart,Mich...",Christopher Nolan,"Jonathan Nolan,Christopher Nolan,David S. Goyer"
3,4,The Godfather Part II,1974,9.0,"Crime,Drama",R,3h 22m,All the power on earth can't change destiny.,13000000,47961919,"Al Pacino,Robert De Niro,Robert Duvall,Diane K...",Francis Ford Coppola,"Francis Ford Coppola,Mario Puzo"
4,5,12 Angry Men,1957,9.0,"Crime,Drama",Approved,1h 36m,Life Is In Their Hands -- Death Is On Their Mi...,350000,955,"Henry Fonda,Lee J. Cobb,Martin Balsam,John Fie...",Sidney Lumet,Reginald Rose


In [ ]:
util.col_header_val(df,config_data)

column name and column length validation passed


1

In [ ]:
print("Columns in CSV file are :",df.columns)
print("Columns in YAML file are ",config_data['columns'])

Columns in CSV file are : Index(['rank', 'name', 'year', 'rating', 'genre', 'certificate', 'run_time',
       'tagline', 'budget', 'box_office', 'casts', 'directors', 'writers'],
      dtype='object')
Columns in YAML file are  ['rank', 'name', 'year', 'rating', 'genre', 'certificate', 'run_time', 'tagline', 'budget', 'box_office', 'casts', 'directors', 'writers']


In [ ]:
if util.col_header_val(df,config_data) == 1:
    print("Validation Pass")
else:
    print("Validation Failed")

column name and column length validation passed
Validation Pass
